In [37]:
import pandas as pd
import numpy as np

In [38]:
# Load the gun violence dataset
df = pd.read_csv('gun_violence_cleaned.csv')

# Display basic information
print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
pd.set_option('display.max_columns', None)
df.head()

Dataset shape: (239677, 43)

First few rows:


,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,num_victims,num_suspects,num_females,num_males,num_adults,num_teens,num_adult_suspects,num_adult_victims,num_teen_suspects,num_teen_victims,num_male_suspects,num_male_victims,num_female_suspects,num_female_victims
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN,4,1,1,1,5,0,1,4,0,0,0,3,1,0
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0,4,1,0,0,4,0,0,4,0,0,0,1,0,0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0,3,2,0,0,5,0,2,3,0,0,2,3,0,0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0,3,1,1,1,4,0,1,3,0,0,1,2,0,1
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0,3,1,2,2,3,1,1,2,0,1,0,2,1,1


In [39]:
# Examine gun_stolen column
print("Sample gun_stolen values:")
print(df['gun_stolen'].value_counts(dropna=False).head(20))

Sample gun_stolen values:
gun_stolen
0::Unknown                                                                                        121310
NaN                                                                                                99498
0::Unknown||1::Unknown                                                                              6116
0::Stolen                                                                                           4503
0::Unknown||1::Unknown||2::Unknown                                                                  1484
0::Not-stolen                                                                                       1352
0::Stolen||1::Stolen                                                                                 719
0::Unknown||1::Unknown||2::Unknown||3::Unknown                                                       583
0::Stolen||1::Unknown                                                                                375
0:Unknown         

In [40]:
# Create a function to count stolen guns
def count_stolen_guns(gun_stolen_str):
    if pd.isna(gun_stolen_str):
        return 0
    # Split by || to get individual guns
    guns = str(gun_stolen_str).split('||')
    # Count how many contain "Stolen" (not "Not-stolen")
    stolen_count = 0
    for gun in guns:
        if '::' in gun:
            status = gun.split('::')[1].strip()
            if status.lower() == 'stolen':
                stolen_count += 1
    return stolen_count

In [41]:
# Create a function to count unknown guns
def count_unknown_guns(gun_stolen_str):
    if pd.isna(gun_stolen_str):
        return 0
    # Split by || to get individual guns
    guns = str(gun_stolen_str).split('||')
    # Count how many contain "Unknown"
    unknown_count = 0
    for gun in guns:
        if '::' in gun:
            status = gun.split('::')[1].strip()
            if status.lower() == 'unknown':
                unknown_count += 1
        elif gun.strip().lower() == '0:unknown':  # Handle the malformed entry
            unknown_count += 1
    return unknown_count

In [42]:
# Sum all guns involved
total_guns = df['n_guns_involved'].sum()
print(f"Total guns involved across all incidents: {total_guns:,.0f}\n")

# Apply the function to create new column
df['n_guns_stolen'] = df['gun_stolen'].apply(count_stolen_guns)

# Display results
print(f"Total guns stolen: {df['n_guns_stolen'].sum():,.0f}")
print(f"\nIncidents with stolen guns: {(df['n_guns_stolen'] > 0).sum():,.0f}")

# Apply the function to create new column
df['n_guns_unknown'] = df['gun_stolen'].apply(count_unknown_guns)

# Display results
print(f"Total guns with unknown stolen status: {df['n_guns_unknown'].sum():,.0f}")
print(f"\nIncidents with unknown gun status: {(df['n_guns_unknown'] > 0).sum():,.0f}")

Total guns involved across all incidents: 192,452

Total guns stolen: 17,610

Incidents with stolen guns: 7,350
Total guns with unknown stolen status: 172,861

Incidents with unknown gun status: 132,435
